In [ ]:
from nocodb_base import fetch_all_rows_from_nocodb, update_row
from file_utils import write_json, write_tsv, read_tsv
import re
import time


In [ ]:
# Configuration
xc_token = ""
# all cloudfront urls
#table_id = "mxp572bg586eppw"
#view_id = "vwy8ovu97qes5d9h"
# all pages
table_id = "mipbzus6tit41os"
view_id = "vw0tbcsv7njdrdel"

merge_table_id = "ml5fr69pkq9zeri"
merge_view_id = "vwn8jjpocrzon8oc"

noco_base_url = ""
headers = {
    'accept': 'application/json',
    'xc-token': xc_token,
}
seed_file = "add_url_lists/xml_export/urls_venues.txt"

In [ ]:
rows_data = fetch_all_rows_from_nocodb(noco_base_url, headers, table_id, view_id)
merge_rows_data = fetch_all_rows_from_nocodb(noco_base_url, headers, merge_table_id, merge_view_id)
#print(rows_data)

In [ ]:
from file_utils import write_json, write_tsv
output_file = "aqnb_url_list.txt"
write_tsv(output_file, rows_data)

In [ ]:
rows_data = read_tsv("aqnb_url_list.txt")

In [ ]:
from urllib.parse import urlparse, urlunparse, quote

def normalize_url(url):
    url = url.strip()
    parsed = urlparse(url)

    scheme = parsed.scheme.lower()
    netloc = parsed.netloc.lower()

    # Lowercase path, strip trailing slash
    path = parsed.path.rstrip('/').lower()

    # Keep query string if present (as-is)
    query = parsed.query

    # Rebuild the normalized URL
    normalized = urlunparse((scheme, netloc, path, '', query, ''))

    # Encode safely (optional but helpful if comparing to encoded index)
    return quote(normalized, safe=':/?&=%')

In [ ]:
def update_row_defined_data(noco_base_url, headers, table_id, record_id, update_data):
    """Update the index_check column in NocoDB."""
    request_url = (
        f"{noco_base_url}/api/v2/tables/{table_id}/records"
    )
    try:
        response = requests.patch(request_url, headers=headers, json=update_data)
        if response.status_code == 200:
            pass
            #print(f"Updated record {record_id} -> index_check: {index_status}")
        else:
            print(f"Failed to update record {record_id}. Status: {response.status_code}")
            print(f"Response text: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error updating record {record_id}: {e}")

In [ ]:
def create_row_defined_data(noco_base_url, headers, table_id, update_data):
    request_url = (
            f"{noco_base_url}/api/v2/tables/{table_id}/records"
    )
    
    try:
        response = requests.post(request_url, headers=headers, json=update_data)
        if response.status_code == 200:
            pass
            #print(f"created record -> new value")
        else:
            print(f"Failed to create. Status: {response.status_code}")
            log_lines.append(f"failed: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error creating record: {e}") 
        log_lines.append(f"Error: {e}")   

In [ ]:
def merge_multiselect_field(existing, new):
    # Handle both strings and lists gracefully
    def to_set(val):
        if isinstance(val, list):
            return set(val)
        elif isinstance(val, str):
            return set(item.strip() for item in val.split(',') if item.strip())
        else:
            return set()

    current_items = to_set(existing)
    new_items = to_set(new)
    
    combined_items = sorted(current_items.union(new_items))
    return combined_items  # ✅ return as list


In [ ]:
merge_url_lookup = {}
for row in merge_rows_data:
    normalized = normalize_url(row.get("URL"))  # or use "normalized_url" if it's already normalized in table
    merge_url_lookup[normalized] = row

In [ ]:
import requests

merged_rows = {}

# Open log file at the top so we can write and flush progressively
with open("merge_log.txt", "w", encoding="utf-8") as log_file:

    # First, load existing rows from merge table
    for row in merge_rows_data:
        url = row.get("URL")
        if url:
            normalized = normalize_url(url)
            merged_rows[normalized] = row.copy()

    # Now, merge data from input table
    for row in rows_data:
        input_url = row.get("URL")
        if not input_url:
            continue

        normalized = normalize_url(input_url)
        identified_via_new = row.get("Identified_via") or ""
        type_value_new = row.get("type") or ""

        # Safe string logging
        log_file.write(f"----------------------------- item\n")
        log_file.write(f"URL: {normalized}\n")
        log_file.write(f"identified_via_new: {identified_via_new}\n")
        log_file.write(f"type_value_new: {type_value_new}\n")
        log_file.flush()

        if normalized in merged_rows:
            merged = merged_rows[normalized]
            type_existing = merged.get("type", "")
            identified_via_existing = merged.get("Identified_via", "")

            log_file.write(f"----------- existing data\n")
            log_file.write(f"type_existing: {type_existing}\n")
            log_file.write(f"identified_via_existing: {identified_via_existing}\n")
            log_file.flush()

            identified_via_combined = merge_multiselect_field(identified_via_existing, identified_via_new)
            type_combined = merge_multiselect_field(type_existing, type_value_new)

            merged["Identified_via"] = identified_via_combined
            merged["type"] = type_combined

            log_file.write(f"----------- merged data\n")
            log_file.write(f"identified_via_combined: {', '.join(identified_via_combined)}\n")
            log_file.write(f"type_combined: {', '.join(type_combined)}\n")
            log_file.flush()

        else:
            # New entry
            merged_rows[normalized] = {
                "URL": normalized,
                "Identified_via": identified_via_new,
                "type": type_value_new,
            }

    # Upload merged rows
    for row in merged_rows.values():
        log_file.write(f"----------- uploading: {row['URL']}\n")
        log_file.flush()

        create_row_defined_data(noco_base_url, headers, merge_table_id, row)

print("✅ Log written and data uploaded.")
